In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
import joblib


In [2]:
# Chargement des données brutes
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

# On garde uniquement les lignes du train qui ont un score `sii`
train = train.dropna(subset=["sii"]).copy()
train["sii"] = train["sii"].astype(int)

In [3]:
# Colonnes numériques
num_cols = [
    "Basic_Demos-Age",
    "Physical-BMI",
    "PreInt_EduHx-computerinternet_hoursday",
    "SDS-SDS_Total_T"
]

# Colonnes catégorielles simples
cat_cols = [
    "Basic_Demos-Sex",                 # 0 = Garçon, 1 = Fille
    "Basic_Demos-Enroll_Season",       # Saison d'inscription
    "Physical-Season"                  # Saison des mesures physiques
]

# Sélection des colonnes + la cible
features = num_cols + cat_cols
target = "sii"

# ⚙️ Création des X, y
X = train[features].copy()
y = train[target]
X_test = test[features].copy()


In [4]:
print("🔍 NaN dans X :\n", X.isnull().sum())

🔍 NaN dans X :
 Basic_Demos-Age                             0
Physical-BMI                              209
PreInt_EduHx-computerinternet_hoursday     82
SDS-SDS_Total_T                           211
Basic_Demos-Sex                             0
Basic_Demos-Enroll_Season                   0
Physical-Season                           141
dtype: int64


In [5]:
# Séparation numérique / catégoriel
num_cols = ["Basic_Demos-Age", "Physical-BMI", "PreInt_EduHx-computerinternet_hoursday", "SDS-SDS_Total_T"]
cat_cols = ["Basic_Demos-Sex", "Basic_Demos-Enroll_Season", "Physical-Season"]

# Imputation numérique (médiane)
num_imputer = SimpleImputer(strategy="median")
X[num_cols] = num_imputer.fit_transform(X[num_cols])
X_test[num_cols] = num_imputer.transform(X_test[num_cols])

# Imputation catégorielle (mode)
cat_imputer = SimpleImputer(strategy="most_frequent")
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])
X_test[cat_cols] = cat_imputer.transform(X_test[cat_cols])

# Vérification
print(" Vérif NaN après imputation :")
print(X.isnull().sum())

 Vérif NaN après imputation :
Basic_Demos-Age                           0
Physical-BMI                              0
PreInt_EduHx-computerinternet_hoursday    0
SDS-SDS_Total_T                           0
Basic_Demos-Sex                           0
Basic_Demos-Enroll_Season                 0
Physical-Season                           0
dtype: int64


In [6]:
# On encode chaque colonne catégorielle une par une
encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    X_test[col] = le.transform(X_test[col])  # attention : le test doit suivre l’ordre du train
    encoders[col] = le  # on les garde si besoin de décoder plus tard

print("Encodage terminé.")
print(X[cat_cols].head())

Encodage terminé.
   Basic_Demos-Sex  Basic_Demos-Enroll_Season  Physical-Season
0                0                          0                0
1                0                          2                0
2                1                          2                0
3                0                          3                2
5                1                          1                2


In [7]:
# ⚖️ Équilibrage + split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 🚀 Modèle LightGBM simple
model = LGBMClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=1 
)
model.fit(X_train, y_train)

# 📈 Prédictions
y_pred = model.predict(X_val)

# 🧪 Évaluation
qwk = cohen_kappa_score(y_val, y_pred, weights='quadratic')
print(f"✅ LightGBM - QWK Score : {qwk:.4f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000678 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 334
[LightGBM] [Info] Number of data points in the train set: 2188, number of used features: 7
[LightGBM] [Info] Start training from score -0.540042
[LightGBM] [Info] Start training from score -1.320842
[LightGBM] [Info] Start training from score -1.980316
[LightGBM] [Info] Start training from score -4.394906
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
✅ LightGBM - QWK Score : 0.3258


## Conclusion – Modèle V2 avec LightGBM

Dans cette version, on a enrichi notre modèle en :

- ajoutant des **variables catégorielles pertinentes** (`Sex`, `Season`, etc.)
- utilisant un modèle plus puissant : **LightGBM**

Résultat : un score **QWK de 0.3258**, meilleur que notre baseline RandomForest (0.3088)

Cela montre que des choix simples et logiques peuvent **améliorer significativement la qualité du modèle**, même sans tuning avancé.

prochaine étape possible : optimiser les hyperparamètres et tester plus de features.


In [8]:
# Modèle LightGBM de base
lgbm = LGBMClassifier(random_state=42, n_jobs=1)

# Grille d'hyperparamètres
param_grid = {
    'num_leaves': [15, 31, 50],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [100, 200]
}

# GridSearch avec scoring personnalisé
grid = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring='roc_auc_ovr',  # approximation pour multiclass
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid.fit(X, y)
print(" Best params:", grid.best_params_)

Fitting 3 folds for each of 54 candidates, totalling 162 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000065 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 337
[LightGBM] [Info] Number of data points in the train set: 2736, number of used features: 7
[LightGBM] [Info] Start training from score -0.540250
[LightGBM] [Info] Start training from score -1.321208
[LightGBM] [Info] Start training from score -1.979358
[LightGBM] [Info] Start training from score -4.387892
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [9]:
# 🌟 Meilleurs paramètres du GridSearch
best_params = grid.best_params_
print("Meilleurs paramètres trouvés :", best_params)

# 🔥 Entraînement final avec les meilleurs paramètres
final_model = LGBMClassifier(**best_params, random_state=42, n_jobs=1)
final_model.fit(X_train, y_train)

# 📈 Prédictions
y_pred = final_model.predict(X_val)

# 🧪 Évaluation
qwk = cohen_kappa_score(y_val, y_pred, weights='quadratic')
print(f" QWK score avec optimisation : {qwk:.4f}")


Meilleurs paramètres trouvés : {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'num_leaves': 15}
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000070 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 334
[LightGBM] [Info] Number of data points in the train set: 2188, number of used features: 7
[LightGBM] [Info] Start training from score -0.540042
[LightGBM] [Info] Start training from score -1.320842
[LightGBM] [Info] Start training from score -1.980316
[LightGBM] [Info] Start training from score -4.394906
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

In [14]:
# Sauvegarder le modèle LightGBM
joblib.dump(final_model, '../models/model_lightgbm.pkl')


['../models/model_lightgbm.pkl']